# Custom MicroPython

The WiFi co-processor requires two custom C modules:

* [**MessagePack**](https://msgpack.org/) is an efficient binary serializer available for many languages, and
* **FinaliserProxy**, a helper class for MicroPython to call the finaliser of user-defined Python objects.

Let's add them to the stock MicroPython interpreter.

The instructions below are for an `esp32` wifi server (called `wifi-server`) and an `stm32` client (`wifi-client`).

## Server

Install/update the local MicroPython branch:

In [1]:
%%bash

# interpreter
cd $IOT/mp
if [ ! -d micropython ]
then
    git clone git@github.com:micropython/micropython.git
else
    cd micropython
    git checkout master
    git pull
    git merge master
fi

# library
cd $IOT/mp
if [ ! -d micropython-lib ]
then
    git clone git@github.com:micropython/micropython-lib.git
else
    cd micropython-lib
    git checkout master
    git pull
    git merge master
fi

Already on 'master'
Your branch is up to date with 'origin/master'.
Already up to date.
Already up to date.
Cloning into 'micropython-lib'...


Find the port of the connected microcontroller (for flashing):

In [1]:
%connect wifi-server

Connected to wifi-server @ serial:///dev/ttyUSB0


Update the port (last line) in the cell below, then run it compile and flash the custom MicroPython interpreter with the new modules:

In [1]:
%%service esp-idf

cd $IOT/mp/micropython/ports/esp32
make submodules
make BOARD=GENERIC_OTA clean 
make BOARD=GENERIC_OTA \
     FROZEN_MANIFEST=../../../boards/manifest_release.py \
     USER_C_MODULES=../../../../modules/micropython.cmake \
     PORT=/dev/ttyUSB0 deploy

setting up IDF ...
idf.py -D MICROPY_BOARD=GENERIC_OTA -B build-GENERIC_OTA  -DUSER_C_MODULES=../../../../modules/micropython.cmake -D MICROPY_FROZEN_MANIFEST=../../../boards/manifest_release.py -p /dev/ttyUSB0 -b 460800 flash
[1/216] cd /home/iot/iot49.org/mp/micropython/ports/esp32/build-GENERIC_OTA/esp-idf/main && echo -n
[2/215] Performing build step for 'bootloader'
ninja: no work to do.
[3/213] Generating ../../genhdr/moduledefs.h
[4/213] Generating ../../genhdr/qstr.i.last
[5/213] Generating ../../genhdr/qstr.split
[6/213] Generating ../../genhdr/qstrdefs.collected.h
QSTR not updated
[7/14] Building C object esp-idf/main/CMakeFiles/__idf_main.dir/home/iot/iot49.org/mp/micropython/py/objmodule.c.obj
[8/14] Building C object esp-idf/main/CMakeFiles/__idf_main.dir/home/iot/iot49.org/mp/modules/ioctl/ioctl.c.obj
/home/iot/iot49.org/mp/modules/ioctl/ioctl.c: In function 'ioctl':
/home/iot/iot49.org/mp/modules/ioctl/ioctl.c:39:57: warning: passing argument 3 of 'stream_p->ioctl' makes

## Client

Now let's add msgpack to the client. In this example I am using an [STM32F405 from Adafruit](https://www.adafruit.com/product/4382). 

The default board description that comes with MicroPython, ADAFRUIT_F405_EXPRESS, stores the file system in the internal flash of the processor with limited space available. The custom version, ADAFRUIT_F405_IOT, instead uses the external QSPI flash with 2MBytes capacity. It also disables including the `usocket` and `network` modules which the wifi coprocessor software replaces.

If you use a different processor update the BOARD variable.

In [1]:
%%service arm32

cd $IOT/mp/micropython/ports/stm32
cp -rf ../../../boards/ADAFRUIT_F405_IOT boards
make submodules
make BOARD=ADAFRUIT_F405_IOT clean
make BOARD=ADAFRUIT_F405_IOT \
     USER_C_MODULES=../../../modules

Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
Updating submodules: lib/libhydrogen lib/lwip lib/mbedtls lib/stm32lib
Synchronizing submodule url for '../../lib/libhydrogen'
Synchronizing submodule url for '../../lib/lwip'
Synchronizing submodule url for '../../lib/mbedtls'
Synchronizing submodule url for '../../lib/stm32lib'
Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
rm -rf build-ADAFRUIT_F405_IOT 
Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
Including User C Module from ../../../modules/finaliserproxy
Including User C Module from ../../../modules/msgpack
mkdir -p build-ADAFRUIT_F405_IOT/genhdr
GEN build-ADAFRUIT_F405_IOT/genhdr/pins.h
GEN stmconst build-ADAFRUIT_F405_IOT/modstm_qstr.h
GEN build-ADAFRUIT_F405_IOT/genhdr/pybcdc.inf
GEN build-ADAFRUIT_F405_IOT/genhdr/pybcdc_inf.h
GEN build-ADAFRUIT_F405_IOT/genhdr/pllfreqtable.h
GEN build-ADAFRUIT_F405_IOT/genhdr/mpversion.h

Flash the custom interpreter to the target board.

## Verify

Now check out the new modules on the wifi server and client.

### MessagePack

An example ... note that the packed version is quite a bit smaller than the text representation. Msgpack also handles more types than json, for example bytes.

In [1]:
%%connect wifi-server wifi-client

import msgpack, json
from io import BytesIO

# msgpack takes a "stream-like" target
buf = BytesIO()

# object to pack
obj = {'list': [True, False, None, 1, 'abc'], 'str': 'blah', 'bytes': b'012345'}

msgpack.pack(obj, buf)
print("packed   [{}]: {}".format(len(buf.getvalue()), buf.getvalue()))

# unpack
buf.seek(0)
obj_unpacked = msgpack.unpack(buf)
print("unpacked [{}]: {}".format(len(repr(obj)), obj_unpacked))


----- wifi-server

packed   [38]: b'\x83\xa3str\xa4blah\xa5bytes\xc4\x06012345\xa4list\x95\xc3\xc2\xc0\x01\xa3abc'
unpacked [74]: {'bytes': b'012345', 'list': [True, False, None, 1, 'abc'], 'str': 'blah'}

----- wifi-client

packed   [38]: b'\x83\xa3str\xa4blah\xa5bytes\xc4\x06012345\xa4list\x95\xc3\xc2\xc0\x01\xa3abc'
unpacked [74]: {'bytes': b'012345', 'list': [True, False, None, 1, 'abc'], 'str': 'blah'}


A more sophisticated example that makes use of MessagePack's custom types to serialize `MyClass`:

In [1]:
%%connect wifi-server wifi-client

from msgpack import pack, unpack, ExtType
from io import BytesIO

class MyClass:
    def __init__(self, val):
        self.value = val
    def __str__(self):
        return str(self.value)

def encoder(obj):
    if isinstance(obj, MyClass):
        # represent MyClass as object with code == 1
        # valid codes are in the range 0~127
        return ExtType(1, obj.value)
    return "no encoder for {}".format(obj)

def decoder(code, data):
    if code == 1:
        # code == 1 is our code for MyClass
        return MyClass(data)
    return "no decoder for type {}".format(code)

data = MyClass(b'my_value')

buffer = BytesIO()
pack(data, buffer, default=encoder)
buffer.seek(0)
decoded = unpack(buffer, ext_hook=decoder)
print("{} -> {} -> {}".format(data, buffer.getvalue(), decoded))


----- wifi-server

b'my_value' -> b'\xd7\x01my_value' -> b'my_value'

----- wifi-client

b'my_value' -> b'\xd7\x01my_value' -> b'my_value'


RPC uses the encoder on the server and the decoder on the client to send "proxies" of classes (e.g. instances of socket) from the esp32 to the client.

### FinaliserProxy

CPython calls `__del__` when the garbage collector sweeps an object. MicroPython implements this feature only for classes written in C.

`FinaliserProxy` is a special class written in C that calls `__del__` of derived classes.

In [1]:
%%connect wifi-server wifi-client

try:
    from finaliserproxy import FinaliserProxy
except ImportError:
    # CPython compatibility (and ports that do not implement this feature)
    print("no finaliser proxy ...")
    class FinaliserProxy:
        def __init__(self, cb):
            pass
    
import gc

class FP(FinaliserProxy):
    def __init__(self, desc):
        self.desc = desc
        super().__init__(self.__del__)
    def __del__(self):
        print("__del__: finalise", self.desc)

for i in range(20):
    f = FP("obj {}".format(i))
    if i % 4 == 0:
        print("--------- collect ...", i)
        gc.collect()

gc.collect()

print("DONE")


----- wifi-server

--------- collect ... 0
__del__: finalise obj 19
--------- collect ... 4
__del__: finalise obj 1
__del__: finalise obj 2
__del__: finalise obj 3
__del__: finalise obj 0
--------- collect ... 8
__del__: finalise obj 5
__del__: finalise obj 6
__del__: finalise obj 7
__del__: finalise obj 4
--------- collect ... 12
__del__: finalise obj 9
__del__: finalise obj 10
__del__: finalise obj 11
__del__: finalise obj 8
--------- collect ... 16
__del__: finalise obj 13
__del__: finalise obj 14
__del__: finalise obj 15
__del__: finalise obj 12
__del__: finalise obj 17
__del__: finalise obj 18
__del__: finalise obj 16
DONE

----- wifi-client

--------- collect ... 0
--------- collect ... 4
__del__: finalise obj 1
__del__: finalise obj 2
__del__: finalise obj 3
__del__: finalise obj 0
--------- collect ... 8
__del__: finalise obj 5
__del__: finalise obj 6
__del__: finalise obj 7
__del__: finalise obj 4
--------- collect ... 12
__del__: finalise obj 9
__del__: finalise obj 10
__del

RPC uses this feature on the wifi client to advise the server when a proxy object has been collected. The server in turn makes the actual object available for garbage collection.